In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
trashpick = pd.read_csv(r"C:\Users\ekims\Documents\NSS\DA14\Projects\trash-pickups-v2-ekimshaf\data\trash_hauler_report_with_lat_lng.csv")
print(trashpick.shape)
trashpick.head(5)

(20226, 13)


,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861


In [3]:
## filter new datafram to include only instances for provider "Red River"
red_river = trashpick[trashpick['Trash Hauler'].isin(['RED RIVER'])]
print(red_river.shape)
red_river.head(5)

(14395, 13)


,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861
5,25317,11/1/2017,Trash Collection Complaint,left trash cart in middle of driveway instead ...,3602 floral dr,37211.0,RED RIVER,4304,16.0,1751660.164,643215.2011,-86.733980,36.098140


In [4]:
## filter new dataframe for only values where description includes the word "miss"
## letter case was ignored, used Description column as Request column had more than one description type associated with missed p/u
rr_missed = red_river[red_river['Request'].str.contains('Trash - Curbside/Alley Missed Pickup', case=False, na=False)]
print(rr_missed.shape)
rr_missed.head(5)


(10715, 13)


,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861
8,25330,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed.,4484 Lavergne Couchville Pike,37013.0,RED RIVER,4210,33.0,1794533.514,618749.3427,-86.588364,36.031728
10,25341,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed trash pickup - resident has at curb for...,3113 HYDES FERRY RD,37218.0,RED RIVER,4204,2.0,1721577.768,676018.3999,-86.836750,36.187584


In [5]:
## counting instances where addresses appear more than once from missed p/u df

address_counts = rr_missed['Incident Address'].value_counts()
rpt_miss = address_counts[address_counts >= 2]

print(address_counts.shape)
print(rpt_miss.shape)
rpt_miss.head(20)

(8190,)
(1551,)


Incident Address
12546 Old Hickory Blvd, Nashville, TN 37013, United States    19
5135 Hickory Hollow Pkwy                                      16
802 Crescent Rd, Nashville, TN 37205, United States           16
607 Estes Rd, Nashville, TN 37215, United States              15
3710 N NATCHEZ CT                                             14
6007 Obrien Ave, Nashville, TN 37209, United States           14
320 Old Hickory Blvd, Nashville, TN 37221, United States      12
111 Barton Ln, Nashville, TN 37214, United States             11
1537 Harding Pl, Nashville, TN 37215, United States           11
3929 Stewarts Ln                                              10
208 Flint Ridge Ct, Nashville, TN 37189, United States         9
2731 Murfreesboro Pike                                         9
115 Barton Ln, Nashville, TN 37214, United States              9
1916 S Hamilton Rd                                             9
1584 Bell Rd                                                   9
14885 ol

In [6]:
## summing the total number of missed incidents with addresses where p/u miss is >= 2
missed_sum = sum(rpt_miss)
missed_sum


4075

In [7]:
## subtracting first instance from counts as it is not subject to fine
## calculated by subtracting count of missed p/u from sum of instances
penalty = missed_sum - rpt_miss.count()
penalty

2524

In [8]:
## calculating total fine, formatting to make it currency
##tot_fine = "${:,.2f}".format(penalty * 200)  <<---- wasn't playing nice with the below calculation because this is making the output a string
tot_fine = penalty * 200
tot_fine


504800

Instruction on checking work:

1. Look at each calculation separately. How many incidents of missed pickups are there? Multiply that times the fee amount. That will tell you the max amount that the fine could be. 2. Look at the count of all house, then the count of all houses with more than one pickup. What percentage is that. If you take your total for above and reduce it by that percentage, do they line up?

In [9]:
## total of all data for missed pick-up
tot_missed = trashpick[trashpick['Request'].str.contains('Trash - Curbside/Alley Missed Pickup', case=False, na=False)]
## total count of missed pick-up addresses
tot_count = tot_missed['Incident Address'].value_counts()
## total number of addresses missed more than once
rpt_miss_all = tot_count[tot_count >= 2]
## penalty for all missed in dataset
all_miss_sum =  sum(rpt_miss_all)
all_pen = all_miss_sum - rpt_miss_all.count()
all_fine = all_pen * 200
##all_fine = "${:,.2f}".format(all_pen * 200)  <<---- wasn't playing nice with the below calculation because this is making the output a string

all_fine

720800

In [10]:
rr_fine_perc = tot_fine / all_fine
rr_fine_perc

0.7003329633740288

In [31]:
all_house = trashpick[trashpick['Request'].str.contains('Trash - Curbside/Alley Missed Pickup', case=False, na=False)]
count_house_all = all_house['Incident Address'].value_counts()
rpt_house_all = count_house_all[count_house_all >= 2]
rpt_house_all
tot_house_all = sum(rpt_house_all)
print(rpt_house_all)
print(tot_house_all)

Incident Address
12546 Old Hickory Blvd, Nashville, TN 37013, United States    19
5135 Hickory Hollow Pkwy                                      17
802 Crescent Rd, Nashville, TN 37205, United States           17
1816 Jo Johnston Ave, Nashville, TN 37203, United States      16
607 Estes Rd, Nashville, TN 37215, United States              15
                                                              ..
3702 Hobbs Rd                                                  2
1114 Saunders Ave, Madison, TN 37115, United States            2
1824 Shackleford Rd B                                          2
4014 Dorcas Dr, Nashville, TN 37215, United States             2
109 Jocelyn Hills Rd                                           2
Name: count, Length: 2226, dtype: int64
5830


In [19]:
rr_house = all_house[all_house['Trash Hauler'].str.contains('RED RIVER', case=False, na=False)]
tot_house_rr = sum(rr_house.value_counts())
tot_house_rr

10668

In [33]:
house_perc = missed_sum / tot_house_all
house_perc

0.6989708404802745

ANSWER:  Total fine for Red River is $504,800.00.

I checked this answer by finding the total fine and calculating the percentage of the total fine that I've calculated is Red River's portion,
which is approx. 70%.

I then found the total number of houses in the dataset which include the "Missed Pick-up" tag with more than one entry in the data set.  I then divided my previous total of Red River missed house >=2 by the total missed >= 2 to again find Red River's portion.  This outcome was again approx. 70%.

This leads me to believe that my orignal calculations were correct in regards to the fine owed by Red River for repeat missed pick-ups.